# Brick optimisation - simple proj rep calc

Created 08/04/2025

Objectives:
* Caclulate proj rep, check it's working so we can put into another object.

# Package imports

In [1]:
import sys
sys.path.append('../')

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

from SPTOptimization.utils import (
    to_npc_array,
    get_right_identity_environment,
    get_right_identity_environment_from_tp_tensor,
    multiply_stacked_unitaries_against_mps,
    inner_product_b_tensors
)

from SPTOptimization.Optimizers.MPSBrickSolver import(
    MPSBrickSolver
)

# Load data

In [5]:
DATA_DIR = r"../data/simple_fermionic_cluster_200_site_dmrg/"

In [6]:
def parse_file_name(file_name):
    non_trivial_proj_rep = bool(int(file_name[3]))

    n1_01 = int(file_name[8])
    n1_10 = int(file_name[9])

    return (non_trivial_proj_rep, (n1_01, n1_10))

In [7]:
loaded_data = dict()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data[data_info]=data['wavefunction']

In [8]:
loaded_data

{(False, (1, 1)): <tenpy.networks.mps.MPS at 0x7d4bb7098c50>,
 (False, (0, 0)): <tenpy.networks.mps.MPS at 0x7d4bb6a2bc50>,
 (True, (1, 0)): <tenpy.networks.mps.MPS at 0x7d4bb66a4e50>,
 (True, (0, 1)): <tenpy.networks.mps.MPS at 0x7d4bb6705610>,
 (True, (0, 0)): <tenpy.networks.mps.MPS at 0x7d4bb6547610>,
 (True, (1, 1)): <tenpy.networks.mps.MPS at 0x7d4bb65ed210>,
 (False, (1, 0)): <tenpy.networks.mps.MPS at 0x7d4bb6806810>,
 (False, (0, 1)): <tenpy.networks.mps.MPS at 0x7d4bb6453710>}

In [9]:
loaded_data[(True, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 8.06502780e-58, 1.23690174e-32, 7.94099304e-32,
       2.05694015e-32, 2.18149273e-32, 2.36122489e-31, 1.92812549e-29,
       1.02382538e-31, 6.43426399e-32, 5.43108385e-32, 1.44142395e-32,
       9.50700006e-33, 3.34253978e-32, 2.22062291e-32, 3.11301804e-32,
       1.59644622e-32, 3.18549623e-32, 8.97734403e-33, 2.18002233e-32,
       7.68854991e-33, 1.96149649e-31, 6.36170358e-32, 1.29773074e-31,
       8.45714546e-33, 5.35962791e-32, 2.38291158e-34, 3.21235332e-32,
       4.13468673e-30, 2.47075839e-32, 3.86189285e-32, 8.12728123e-33,
       1.43713253e-32, 1.17453463e-32, 6.02315024e-32, 1.88833724e-32,
       2.12686593e-30, 1.19221600e-31, 4.16532767e-32, 1.64792348e-29,
       2.02150553e-30, 9.03437657e-32, 1.34399706e-32, 2.29796943e-32,
       1.67067027e-31, 2.62462347e-32, 8.24426744e-33, 6.51166714e-33,
       2.71371928e-32, 2.06626976e-32, 2.89736867e-31, 1.45535890e-31,
       1.13467738e-30, 2.00315369e-32, 1.62325755e-32, 1.40365954e-29,
      

In [10]:
loaded_data[(False, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 0.00000000e+00, 3.59424750e-29, 7.59555955e-29,
       1.25737033e-28, 7.59555955e-29, 6.65878723e-29, 3.59424750e-29,
       0.00000000e+00, 1.14662165e-29, 0.00000000e+00, 6.56849963e-29,
       6.56849963e-29, 7.49910898e-29, 8.98561875e-30, 3.59424750e-29,
       3.59424750e-29, 5.43574468e-30, 8.98561875e-30, 3.59424750e-29,
       5.43574468e-30, 1.14662165e-29, 3.59424750e-29, 3.59424750e-29,
       2.95946099e-29, 1.25737033e-28, 8.01495006e-30, 6.56849963e-29,
       3.02016630e-29, 7.49910898e-29, 3.59424750e-29, 1.14662165e-29,
       3.02016630e-29, 3.59424750e-29, 7.49910898e-29, 1.25737033e-28,
       1.39804025e-28, 1.92592994e-28, 0.00000000e+00, 7.70371978e-30,
       1.39804025e-28, 3.59424750e-29, 3.59424750e-29, 7.70371978e-30,
       7.30805673e-29, 3.02016630e-29, 0.00000000e+00, 1.39804025e-28,
       3.59424750e-29, 7.70371978e-30, 3.59424750e-29, 3.59424750e-29,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.59424750e-29,
      

In [11]:
loaded_exps = dict()

for k, psi in loaded_data.items():
    exp = psi.expectation_value('N', sites=[101,])[0]
    loaded_exps[k] = exp

In [12]:
loaded_exps

{(False, (1, 1)): 0.49999999999999667,
 (False, (0, 0)): 3.594247499413394e-29,
 (True, (1, 0)): 0.49999999999998707,
 (True, (0, 1)): 0.5000000000000188,
 (True, (0, 0)): 2.897368673320364e-31,
 (True, (1, 1)): 0.4999999999999868,
 (False, (1, 0)): 0.4999999999999989,
 (False, (0, 1)): 0.5000000000000056}

# Definitons

In [13]:
MAX_VIRTUAL_BOND_DIM = 8
MAX_INTERMEDIATE_VIRTUAL_BOND_DIM = 2*MAX_VIRTUAL_BOND_DIM
# MPO bond dim?
MAX_MPO_BOND_DIM = 50

SVD_CUTOFF = 1e-3

Define bosonic symmetries. Label by the group element added.

In [14]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [15]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [16]:
bosonic_npc_symmetries = [
    to_npc_array(X) for X in bosonic_np_symmetries
]

In [17]:
npc_00 = to_npc_array(np_00)
npc_01 = to_npc_array(np_01)
npc_10 = to_npc_array(np_10)
npc_11 = to_npc_array(np_11)

Define "fermionic symmetries". Just identity and JW string.

In [18]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [19]:
fermionic_np_symmetries = [np_I, np_JW]

In [20]:
fermionic_npc_symmetries = [
    to_npc_array(X) for X in fermionic_np_symmetries
]

In [21]:
npc_JW = fermionic_npc_symmetries[1]

In [22]:
symmetry_actions = [
    [[b, f] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [23]:
shifted_symmetry_actions = [
    [[f, b] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [24]:
cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*50,
                left_symmetry_index=50,
                left_boundary_unitaries=[np_I, np_00]*1,
                right_boundary_unitaries=[np_00, np_I]*1
            )

            cases[(*k, i, j)] = case

In [25]:
for c in cases.values():
    c.compute_svd_approximate_expectation()

In [26]:
shifted_cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(shifted_symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*50,
                left_symmetry_index=51,
                left_boundary_unitaries=[np_00, np_I]*1,
                right_boundary_unitaries=[np_I, np_00]*1
            )

            shifted_cases[(*k, i, j)] = case

In [27]:
for c in shifted_cases.values():
    c.compute_svd_approximate_expectation()

In [28]:
symmetric_cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                [s[1],] + s*49,
                left_symmetry_index=51,
                left_boundary_unitaries=[np_00, np_I, np_00],
                right_boundary_unitaries=[np_00, np_I, np_00]
            )

            symmetric_cases[(*k, i, j)] = case

In [29]:
for c in symmetric_cases.values():
    c.compute_svd_approximate_expectation()

In [30]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [31]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

# Sweep

In [32]:
block_width = 5
num_one_sided_iterations = 1
num_two_sided_iterations = 3
num_layers = 1
num_sites = 5

In [33]:
optimizer_dict = dict()

for k, v in cases.items():
    opt = MPSBrickSolver(
        v,
        num_sites,
        block_width,
        num_layers,
        num_one_sided_iterations,
        num_two_sided_iterations,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    )

    opt.optimise()

    optimizer_dict[k] = opt

In [34]:
final_exps = dict()

for k, v in optimizer_dict.items():
    final_exp = v.flatten_exps()[-1]
    final_exps[k] = final_exp

In [35]:
bad_cases = [
    k
    for k, v in final_exps.items()
    if v < 0.99
]

In [36]:
bad_cases

[]

# Test proj rep

Give a wavefunction, calculate proj rep.

## SPT functions

In [37]:
def calculate_proj_rep_phase(state_g, left_schmidt_values_g, unitary_h,
                             state_gh,
                             left_environment,
                             max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    prod_state, _ = multiply_stacked_unitaries_against_mps(
        unitary_h,
        state_g,
        left_schmidt_values_g,
        max_virtual_bond_dim
    )

    exp = inner_product_b_tensors(prod_state, state_gh, left_environment)

    return exp/(np.abs(exp))

In [38]:
def get_proj_rep_phases(mps_tensors, left_schmidt_values, unitaries,
                        left_environment):
    mps_dict = {
        k: v for k, v in zip(symmetry_labels[1:], mps_tensors)
    }

    schmidt_vals_dict = {
        k: v for k, v in zip(symmetry_labels[1:], left_schmidt_values)
    }

    unitaries_dict = {
        k: v for k, v in zip(symmetry_labels[1:], unitaries)
    }

    out_phases = list()

    for a, b, c in group_products:
        phase = calculate_proj_rep_phase(
            mps_dict[a],
            schmidt_vals_dict[a],
            unitaries_dict[b],
            mps_dict[c],
            left_environment,
            max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
        )

        out_phases.append(phase)

    return out_phases

## Group data definitions

In [39]:
from itertools import combinations

In [40]:
symmetry_labels = [
    'II',
    'IX',
    'XI',
    'XX'
]

In [41]:
symmetry_combination_labels = list(combinations(symmetry_labels, 2))

In [42]:
symmetry_combination_labels

[('II', 'IX'),
 ('II', 'XI'),
 ('II', 'XX'),
 ('IX', 'XI'),
 ('IX', 'XX'),
 ('XI', 'XX')]

In [43]:
e, a, b, c = symmetry_labels

In [44]:
group_products = [
    (a, b, c),
    (b, a, c),
    (a, c, b),
    (c, a, b),
    (c, b, a),
    (b, c, a)
]

## Test case 1

In [45]:
proj_rep = True
n1_pair = (1, 0)

# May need to treat more consistently later.
jw_string = False

In [46]:
sub_opts = [
    optimizer_dict[(proj_rep, n1_pair, jw_string, i)]
    for i in range(4)
]

### Right side

In [47]:
test_opt = sub_opts[0]

In [48]:
mps_tensors = [
    opt.top_right_mps_tensors
    for opt in sub_opts
]

In [49]:
right_side_left_schmidt_values = [
    opt.current_right_side_left_schmidt_values
    for opt in sub_opts
]

In [50]:
unitaries = [
    opt.right_unitaries
    for opt in sub_opts
]

In [51]:
null_opt = optimizer_dict[(proj_rep, n1_pair, 0, 0)]

In [52]:
left_environment = null_opt.right_side_left_symmetry_environment

In [53]:
proj_rep_phases = get_proj_rep_phases(
    mps_tensors,
    right_side_left_schmidt_values,
    unitaries,
    left_environment
)

In [54]:
proj_rep_phases

[(-1+0j),
 (-0.9999999999999999+0j),
 (-1+0j),
 (-0.9999999999999999+0j),
 (-1+0j),
 (-1+0j)]

These are wrong... Wait, need "gauge invariant" phases?

## Test case 2

In [55]:
proj_rep = True
n1_pair = (0, 0)

# May need to treat more consistently later.
jw_string = False

In [56]:
sub_opts = [
    optimizer_dict[(proj_rep, n1_pair, jw_string, i)]
    for i in range(4)
]

### Right side

In [57]:
test_opt = sub_opts[0]

In [58]:
mps_tensors = [
    opt.top_right_mps_tensors
    for opt in sub_opts
]

In [59]:
right_side_left_schmidt_values = [
    opt.current_right_side_left_schmidt_values
    for opt in sub_opts
]

In [60]:
unitaries = [
    opt.right_unitaries
    for opt in sub_opts
]

In [61]:
null_opt = optimizer_dict[(proj_rep, n1_pair, 0, 0)]

In [62]:
left_environment = null_opt.right_side_left_symmetry_environment

In [63]:
proj_rep_phases = get_proj_rep_phases(
    mps_tensors,
    right_side_left_schmidt_values,
    unitaries,
    left_environment
)

In [64]:
proj_rep_phases

[(1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j)]

Better...! So why wrong when I have a non-trivial fermion homology?

## Test case 3

In [65]:
proj_rep = False
n1_pair = (1, 0)

# May need to treat more consistently later.
jw_string = False

In [66]:
sub_opts = [
    optimizer_dict[(proj_rep, n1_pair, jw_string, i)]
    for i in range(4)
]

### Right side

In [67]:
test_opt = sub_opts[0]

In [68]:
mps_tensors = [
    opt.top_right_mps_tensors
    for opt in sub_opts
]

In [69]:
right_side_left_schmidt_values = [
    opt.current_right_side_left_schmidt_values
    for opt in sub_opts
]

In [70]:
unitaries = [
    opt.right_unitaries
    for opt in sub_opts
]

In [71]:
null_opt = optimizer_dict[(proj_rep, n1_pair, 0, 0)]

In [72]:
left_environment = null_opt.right_side_left_symmetry_environment

In [73]:
proj_rep_phases = get_proj_rep_phases(
    mps_tensors,
    right_side_left_schmidt_values,
    unitaries,
    left_environment
)

In [74]:
proj_rep_phases

[(1+0j), (1+0j), (1+0j), (1+0j), (0.9999999999999999+0j), (1+0j)]

## Test case 4

In [75]:
proj_rep = False
n1_pair = (0, 0)

# May need to treat more consistently later.
jw_string = False

In [76]:
sub_opts = [
    optimizer_dict[(proj_rep, n1_pair, jw_string, i)]
    for i in range(4)
]

### Right side

In [77]:
test_opt = sub_opts[0]

In [78]:
mps_tensors = [
    opt.top_right_mps_tensors
    for opt in sub_opts
]

In [79]:
right_side_left_schmidt_values = [
    opt.current_right_side_left_schmidt_values
    for opt in sub_opts
]

In [80]:
unitaries = [
    opt.right_unitaries
    for opt in sub_opts
]

In [81]:
null_opt = optimizer_dict[(proj_rep, n1_pair, 0, 0)]

In [82]:
left_environment = null_opt.right_side_left_symmetry_environment

In [83]:
proj_rep_phases = get_proj_rep_phases(
    mps_tensors,
    right_side_left_schmidt_values,
    unitaries,
    left_environment
)

In [84]:
proj_rep_phases

[(1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j)]

# String order parameters

In [85]:
np_Z_01 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, -1]
])

np_Z_10 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, -1]
])

np_Z_11 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1]
])

In [86]:
bosonic_np_phase_symmetries = [
    np_00,
    np_Z_01,
    np_Z_10,
    np_Z_11
]

In [87]:
bosonic_npc_phase_symmetries = [
    to_npc_array(X) for X in bosonic_np_phase_symmetries
]

In [88]:
npc_Z_01 = to_npc_array(np_Z_01)
npc_Z_10 = to_npc_array(np_Z_10)
npc_Z_11 = to_npc_array(np_Z_11)

In [89]:
fermion_identity = fermionic_npc_symmetries[0]

In [90]:
symmetry_term = [npc_01, fermion_identity]*50

In [91]:
string_order_term_1 = [npc_00, fermion_identity] + symmetry_term + [npc_00,]
string_order_term_2 = [npc_Z_01, fermion_identity] + symmetry_term + [npc_Z_01,]

In [92]:
string_order_params = dict()

n = 0

for l, v in loaded_data.items():
    for i, s in enumerate(bosonic_npc_symmetries):
        for j, b1 in enumerate(bosonic_npc_phase_symmetries):
            for k, b2 in enumerate(bosonic_npc_phase_symmetries):

                term = [b1, fermion_identity] + [s, fermion_identity]*5 + [b2,]
                exp = v.expectation_value_multi_sites(term, 50)

                string_order_params[(*l, i, j, k)] = exp
                n+=1

In [93]:
large_string_order_params = {
    k: v
    for k, v in string_order_params.items()
    if v > 0.1
}

In [94]:
large_string_order_params

{(False, (1, 1), 0, 0, 0): array(1.),
 (False, (1, 1), 3, 0, 0): array(1.),
 (False, (0, 0), 0, 0, 0): array(1.),
 (False, (0, 0), 1, 0, 0): array(1.),
 (False, (0, 0), 2, 0, 0): array(1.),
 (False, (0, 0), 3, 0, 0): array(1.),
 (True, (1, 0), 0, 0, 0): array(1.),
 (True, (0, 1), 0, 0, 0): array(1.),
 (True, (0, 0), 0, 0, 0): array(1.),
 (True, (1, 1), 0, 0, 0): array(1.),
 (False, (1, 0), 0, 0, 0): array(1.),
 (False, (1, 0), 2, 0, 0): array(1.),
 (False, (0, 1), 0, 0, 0): array(1.),
 (False, (0, 1), 1, 0, 0): array(1.)}

In [95]:
{
    k: v
    for k, v in large_string_order_params.items()
    if not k[0]
}

{(False, (1, 1), 0, 0, 0): array(1.),
 (False, (1, 1), 3, 0, 0): array(1.),
 (False, (0, 0), 0, 0, 0): array(1.),
 (False, (0, 0), 1, 0, 0): array(1.),
 (False, (0, 0), 2, 0, 0): array(1.),
 (False, (0, 0), 3, 0, 0): array(1.),
 (False, (1, 0), 0, 0, 0): array(1.),
 (False, (1, 0), 2, 0, 0): array(1.),
 (False, (0, 1), 0, 0, 0): array(1.),
 (False, (0, 1), 1, 0, 0): array(1.)}

In [96]:
np.all((-np_01 @ np_Z_01 @ np_01) == np_Z_01)

True

In [97]:
np.all((-np_10 @ np_Z_10 @ np_10) == np_Z_10)

True

In [98]:
np.all((np_11 @ np_Z_11 @ np_11) == np_Z_11)

True

In [99]:
np.all((-np_11 @ np_Z_10 @ np_11) == np_Z_10)

True

In [100]:
np.all((-np_11 @ np_Z_01 @ np_11) == np_Z_01)

True

In [101]:
np.all((np_01 @ np_Z_10 @ np_01) == np_Z_10)

True

In [102]:
np.all((np_10 @ np_Z_01 @ np_10) == np_Z_01)

True

In [103]:
string_order_params[(False, (0, 0), 1, 2, 2)]

array(1.57772181e-29)

In [104]:
string_order_params[(True, (0, 0), 1, 2, 2)]

array(-2.4713882e-28)

All looks good...

# Verify SPT phase with environments

In [136]:
test_cases = [
    cases[True, (0, 0), 0, i]
    for i in range(4)
]

In [137]:
test_case = test_cases[0]

In [138]:
test_case

In [139]:
test_case.right_projected_symmetry_state

<npc.Array shape=(4, 4) labels=['vR', 'vR*']>

In [140]:
def multiply_right_environments(env1, env2):
    out = npc.tensordot(
        env1,
        env2,
        [['vR',], ['vR*']]
    )

    return out

In [141]:
from functools import reduce

In [142]:
help(reduce)

Help on built-in function reduce in module _functools:

reduce(...)
    reduce(function, iterable[, initial]) -> value
    
    Apply a function of two arguments cumulatively to the items of a sequence
    or iterable, from left to right, so as to reduce the iterable to a single
    value.  For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) calculates
    ((((1+2)+3)+4)+5).  If initial is present, it is placed before the items
    of the iterable in the calculation, and serves as a default when the
    iterable is empty.



In [143]:
def normalise_scaled_unitary(matrix):
    m = matrix

    tr = npc.tensordot(
        m,
        m.conj(),
        [['vR', 'vR*'], ['vR*', 'vR']]
    )

    dim = m.to_ndarray().shape[0]

    scale = np.sqrt(tr/dim)

    out = m/scale

    return out

In [144]:
def right_proj_rep_invariant_phase_from_environments(case0, case1, case2):
    v0 = case0.right_projected_symmetry_state

    v1 = case1.right_projected_symmetry_state
    v1 = npc.tensordot(
        v0.conj(),
        v1,
        [['vR',], ['vR*']]
    )
    v1 = normalise_scaled_unitary(v1)

    #print(np.round(npc.tensordot(v1, v1.conj(), [['vR',], ['vR*']]).to_ndarray()), 2)

    v2 = case2.right_projected_symmetry_state
    v2 = npc.tensordot(
        v0.conj(),
        v2,
        [['vR',], ['vR*']]
    )
    v2 = normalise_scaled_unitary(v2)

    #print(np.round(npc.tensordot(v2, v2.conj(), [['vR',], ['vR*']]).to_ndarray()), 2)

    t = reduce(
        multiply_right_environments,
        [v1, v2, v1.conj(), v2.conj()]
    )

    exp = npc.trace(t)
    phase = exp/np.abs(exp)

    return phase

In [145]:
right_proj_rep_invariant_phase_from_environments(
    test_cases[0],
    test_cases[1],
    test_cases[2]
)

(1+0j)

In [115]:
from itertools import combinations

In [116]:
case_pairs = list(combinations(test_cases[1:], 2))

In [117]:
phases = [
    right_proj_rep_invariant_phase_from_environments(test_cases[0], c1, c2)
    for c1, c2 in case_pairs
]

In [118]:
phases

[(1+0j), (1+0j), (1+0j)]

In [119]:
test_cases = [
    cases[False, (0, 0), 0, i]
    for i in range(4)
]

In [120]:
case_pairs = list(combinations(test_cases[1:], 2))

In [121]:
phases = [
    right_proj_rep_invariant_phase_from_environments(test_cases[0], c1, c2)
    for c1, c2 in case_pairs
]

In [122]:
phases

[(1+0j), (1+0j), (1+0j)]

In [123]:
test_cases = [
    cases[True, (1, 0), 0, i]
    for i in range(4)
]

In [124]:
case_pairs = list(combinations(test_cases[1:], 2))

In [125]:
phases = [
    right_proj_rep_invariant_phase_from_environments(test_cases[0], c1, c2)
    for c1, c2 in case_pairs
]

In [126]:
phases

[(1+0j), (1+0j), (1+0j)]

In [127]:
def right_proj_rep_invariant_phase_from_environments_1(case1, case2):
    v1 = case1.right_projected_symmetry_state
    v1 = normalise_scaled_unitary(v1)

    #print(np.round(npc.tensordot(v1, v1.conj(), [['vR',], ['vR*']]).to_ndarray()), 2)

    v2 = case2.right_projected_symmetry_state

    v2 = normalise_scaled_unitary(v2)

    #print(np.round(npc.tensordot(v2, v2.conj(), [['vR',], ['vR*']]).to_ndarray()), 2)

    t = reduce(
        multiply_right_environments,
        [v1, v2, v1.conj(), v2.conj()]
    )

    exp = npc.trace(t)
    phase = exp/np.abs(exp)

    return phase

In [128]:
test_cases = [
    cases[True, (0, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

In [129]:
test_cases = [
    cases[False, (0, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

In [130]:
test_cases = [
    cases[True, (1, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

In [131]:
test_cases = [
    shifted_cases[True, (0, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

In [132]:
test_cases = [
    shifted_cases[False, (0, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

In [133]:
test_cases = [
    shifted_cases[True, (1, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

Explicitly multiply vectors.

In [146]:
test_cases = [
    cases[True, (0, 0), 0, i]
    for i in range(4)
]

In [147]:
case1, case2, case3 = test_cases[1:]

In [154]:
v1 = case1.right_projected_symmetry_state
v1 = normalise_scaled_unitary(v1)

v2 = case2.right_projected_symmetry_state
v2 = normalise_scaled_unitary(v2)

v12 = multiply_right_environments(v1, v2)
v21 = multiply_right_environments(v2, v1)

v3 = case3.right_projected_symmetry_state
v3 = normalise_scaled_unitary(v3)

In [155]:
v12, v21, v3

(<npc.Array shape=(4, 4) labels=['vR*', 'vR']>,
 <npc.Array shape=(4, 4) labels=['vR*', 'vR']>,
 <npc.Array shape=(4, 4) labels=['vR', 'vR*']>)

In [157]:
X12 = v12.to_ndarray()

In [158]:
X21 = v21.to_ndarray()

In [159]:
X3 = v3.to_ndarray()

In [160]:
X3/X12

array([[-1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j],
       [-1.+0.j, -1.+0.j, -1.+0.j, -1.-0.j],
       [-1.+0.j, -1.+0.j, -1.-0.j, -1.+0.j],
       [-1.+0.j, -1.-0.j, -1.+0.j, -1.+0.j]])

In [161]:
X3/X21

array([[-1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j],
       [-1.+0.j, -1.+0.j, -1.+0.j, -1.-0.j],
       [-1.+0.j, -1.+0.j, -1.-0.j, -1.+0.j],
       [-1.+0.j, -1.-0.j, -1.+0.j, -1.+0.j]])

[Compare with ZXZ model](./../optimisation/cluster_state_virtual_vector_proj_rep_check.ipynb)

So these results are definitely wrong.

# Check entanglement spectrum

In [134]:
for k, v in loaded_data.items():
    print(f"{k} - {v.get_SR(101)}")

(False, (1, 1)) - [0.70710678 0.70710678]
(False, (0, 0)) - [1.]
(True, (1, 0)) - [0.5 0.5 0.5 0.5]
(True, (0, 1)) - [0.5 0.5 0.5 0.5]
(True, (0, 0)) - [0.5 0.5 0.5 0.5]
(True, (1, 1)) - [0.5 0.5 0.5 0.5]
(False, (1, 0)) - [0.70710678 0.70710678]
(False, (0, 1)) - [0.70710678 0.70710678]


In [135]:
for k, v in loaded_data.items():
    print(f"{k} - {v.get_SR(100)}")

(False, (1, 1)) - [0.70710678 0.70710678]
(False, (0, 0)) - [1.]
(True, (1, 0)) - [0.5 0.5 0.5 0.5]
(True, (0, 1)) - [0.5 0.5 0.5 0.5]
(True, (0, 0)) - [0.5 0.5 0.5 0.5]
(True, (1, 1)) - [0.5 0.5 0.5 0.5]
(False, (1, 0)) - [0.70710678 0.70710678]
(False, (0, 1)) - [0.70710678 0.70710678]


So there is something going on... The 2-fold degeneracy due to the fermion spt is unexpected for me.

# To-do:
Calculate string order parameters.